In [1]:
import os
import nibabel as nib
from scipy.ndimage import zoom

def resample_image(input_image_path, output_image_path, new_shape, interpolation_order=3):
    # 加载原始图像
    img = nib.load(input_image_path)
    img_data = img.get_fdata()
    
    # 确保 new_shape 与 img_data 的维度匹配
    if len(new_shape) < img_data.ndim:
        # 如果 new_shape 维度小于原始图像维度，扩展 new_shape
        new_shape = (*new_shape,) + (1,) * (img_data.ndim - len(new_shape))
    elif len(new_shape) > img_data.ndim:
        # 如果 new_shape 维度大于原始图像维度，截断 new_shape
        new_shape = new_shape[:img_data.ndim]

    # 计算新的体素尺寸
    zoom_factors = [new_dim / float(orig_dim) for orig_dim, new_dim in zip(img_data.shape, new_shape)]
    
    # 调整图像大小
    resampled_img_data = zoom(img_data, zoom_factors, order=interpolation_order)
    
    # 创建新的 NIfTI 图像
    resampled_img = nib.Nifti1Image(resampled_img_data, img.affine, img.header)
    
    # 保存调整大小后的图像
    nib.save(resampled_img, output_image_path)

def batch_resize_nii(input_dir, output_dir, new_shape=(64, 256, 256)):
    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 遍历输入目录
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_dir)
                output_path = os.path.join(output_dir, relative_path)
                
                # 确保输出子目录存在
                os.makedirs(output_path, exist_ok=True)
                
                # 调整图像大小并保存
                resample_image(input_path, os.path.join(output_path, file), new_shape)

# 输入和输出目录
input_directory = '/hy-tmp/test-9/'
output_directory = '/hy-tmp/Abeta_4D_resize2/'


# 调用函数
batch_resize_nii(input_directory, output_directory)

KeyboardInterrupt: 